# TODO
- Fix encoding issues with place names table (see below for troublesome records)
- Add remaining geographic hierarchy (Health Regions, CT, DA, DB, ADA, HCCSS)
- Read geographic hierachy from Parquet files and do the SQL work using DuckDB
- Add field so user can search by province (if possible). It won't be possible to add the field to the country and region tables
- Add field so user can search by census year
- Standardize search values. Look into porting CASK into JavaScript as the user input will need to be standardized as well

In [33]:
import os
import sqlite3

from dotenv import load_dotenv
import duckdb
import pandas as pd

# Create the geographies table

## Create tables in SQLite
These are the instructions for exporting the database tables and importing into Cloudflare D1. At the moment they are manually done, but I should automate it.

1. Export the geographies table using `sqlite3`
```
sqlite3 geography.db
.output ./geographies.sql
.dump geographies
```
2. Remove the `PRAGMA foreign_keys=off`, `BEGIN TRANSACTION` and `COMMIT` parts
3. Remove the `CREATE TABLE geographies` statement
4. Add the following to the top, before the insert statements
```
DROP TABLE IF EXISTS geographies;
CREATE TABLE IF NOT EXISTS geographies (
  id INTEGER PRIMARY KEY,
  dguid TEXT,
  search_name TEXT,
  geographic_level INTEGER
);

DROP TABLE IF EXISTS geographies_fts;
CREATE VIRTUAL TABLE IF NOT EXISTS geographies_fts USING fts5(
  id UNINDEXED,
  search_name,
  content='geographies',
  content_rowid='id',
  tokenize = "unicode61 tokenchars '-/.,''&():+'"
);

```
5. Add `INSERT INTO geographies_fts(geographies_fts) VALUES ('rebuild');` at the end of the SQL file
6. Add `PRAGMA optimize;` at the end of the SQL file. This is recommended https://developers.cloudflare.com/d1/best-practices/use-indexes/
7. Log into Cloudflare by doing npx wrangler login
8. Import as follows
```
npx wrangler d1 execute geographies_search --remote --file=./geographies.sql
```

In [49]:
con = sqlite3.connect("geography.db")
cur = con.cursor()

cur.executescript("""
DROP TABLE IF EXISTS geographies;
CREATE TABLE IF NOT EXISTS geographies (
  id INTEGER PRIMARY KEY,
  dguid TEXT,
  search_name TEXT,
  geographic_level INTEGER
);
""")

# Allow searches to use -/.,'&():+
cur.executescript("""
DROP TABLE IF EXISTS geographies_fts;
CREATE VIRTUAL TABLE IF NOT EXISTS geographies_fts USING fts5(
  id UNINDEXED,
  search_name,
  content='geographies',
  content_rowid='id',
  tokenize = "unicode61 tokenchars '-/.,''&():+'"
);
""")

con.commit()

## SQL to create search table
For tables where there is an English and French field, it creates two records. Can probably add a field to the search table that tells the user whether the field is English, French, or Both.

Statistics Canada searches English field when the page is in English, and it searches the French field when the page is in French. Here are the two examples:
- **English:** https://www150.statcan.gc.ca/n1/en/geo?geotext=Quebec%20%5BProvince%5D&geocode=A000224
- **French:** https://www150.statcan.gc.ca/n1/fr/geo?geotext=Qu%C3%A9bec%20%5BProvince%5D&geocode=A000224

In [ ]:
duck_con = duckdb.connect()
duck_con.install_extension("spatial")
duck_con.load_extension("spatial")

duck_con.sql("""
DROP TABLE IF EXISTS geography;
CREATE TABLE geography AS
WITH country AS (
	SELECT country_dguid AS dguid, country_en_name AS search_name, 13 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/country_2021.parquet'
), regions AS (
	SELECT DISTINCT grc_dguid AS dguid, grc_en_name AS search_name, 12 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/grc_2021.parquet'
	UNION
	SELECT DISTINCT grc_dguid AS dguid, grc_fr_name AS search_name, 12 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/grc_2021.parquet'
), pr AS (
	SELECT DISTINCT pr_dguid AS dguid, pr_en_name AS search_name, 11 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'hhttps://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/pr_2021.parquet'
	UNION
	SELECT DISTINCT pr_dguid AS dguid, pr_fr_name AS search_name, 11 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'hhttps://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/pr_2021.parquet'
), er AS (
	SELECT DISTINCT er_dguid AS dguid, er_name AS search_name, 10 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/er_2021.parquet'
), car AS (
	SELECT DISTINCT car_dguid AS dguid, car_en_name AS search_name, 9 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/car_2021.parquet'
	UNION
	SELECT DISTINCT car_dguid AS dguid, car_fr_name AS search_name, 9 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/car_2021.parquet'
), cd AS (
	SELECT cd_dguid AS dguid, cd_name AS search_name, 8 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/cd_2021.parquet'
), ccs AS (
	SELECT ccs_dguid AS dguid, ccs_name AS search_name, 7 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/ccs_2021.parquet'
), cma AS (
	SELECT 
	CASE 
		WHEN cma_p_dguid IS NOT NULL THEN cma_p_dguid
		ELSE cma_dguid 
	END AS dguid, cma_name AS search_name, 6 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/cma_2021.parquet'
), csd AS (
	SELECT csd_dguid AS dguid, csd_name AS search_name, 5 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/csd_2021.parquet'
), fed AS (
	SELECT DISTINCT fed_dguid AS dguid, fed_en_name AS search_name, 4 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/fed_2021_2013.parquet'
	UNION
	SELECT DISTINCT fed_dguid AS dguid, fed_fr_name AS search_name, 4 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/fed_2021_2013.parquet'
), dpl AS (
	SELECT dpl_dguid AS dguid, dpl_name AS search_name, 3 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/dpl_2021.parquet'
), pc AS (
	SELECT 
	CASE 
		WHEN pop_ctr_p_dguid IS NOT NULL THEN pop_ctr_p_dguid
		ELSE pop_ctr_dguid
	END AS dguid, pop_ctr_name AS search_name, 2 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'hhttps://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/pop_ctr_2021.parquet'
), pn AS (
	SELECT pn_dguid AS dguid, pn_name AS search_name, 1 AS geographic_level, ST_AsGeoJSON(geom) AS geom FROM 'https://data.dataforcanada.org/processed/statistics_canada/placenames/2021/pn_2021.parquet'
), concatenation AS (
	SELECT * FROM country
	UNION
	SELECT * FROM regions
	UNION
	SELECT * FROM pr
	UNION
	SELECT * FROM er
	UNION
	SELECT * FROM car
	UNION
	SELECT * FROM cd
	UNION
	SELECT * FROM ccs
	UNION
	SELECT * FROM cma
	UNION
	SELECT * FROM csd
	UNION
	SELECT * FROM fed
	UNION
	SELECT * FROM dpl
	UNION
	SELECT * FROM pc
    UNION
	SELECT * FROM pn
)
SELECT * FROM concatenation
ORDER BY search_name, geographic_level DESC;
""")
duck_con.commit()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
geography = duck_con.sql("SELECT * FROM geography;").df()

# TODO
## Fix encoding issues with place names

In [39]:
dguids_to_fix = ['2021S0515005422',
 '2021S0515007864',
 '2021S0515017557',
 '2021S0515019487',
 '2021S0515019731',
 '2021S0515022795',
 '2021S0515024311',
 '2021S0515028429',
 '2021S0515030028',
 '2021S0515030168',
 '2021S0515030432',
 '2021S0515031197',
 '2021S0515031295',
 '2021S0515031660',
 '2021S0515032370',
 '2021S0515038300',
 '2021S0515038389',
 '2021S0515040448',
 '2021S0515040522']
place_names_to_fix = geography[geography['dguid'].isin(dguids_to_fix)]
place_names_to_fix.head(19)

,dguid,search_name,geographic_level,geom
5653,2021S0515005422,CascapédiaSaint-Jules,1,"{""type"":""Point"",""coordinates"":[-65.9166667,48...."
8243,2021S0515007864,Côte-des-NeigesNotre-Dame-de-Grâce,1,"{""type"":""Point"",""coordinates"":[-73.6263889,45...."
18297,2021S0515017557,L'Île-BizardSainte-Geneviève,1,"{""type"":""Point"",""coordinates"":[-73.866667,45.4..."
20327,2021S0515019487,Le Coteau-des-Surs,1,"{""type"":""Point"",""coordinates"":[-70.456886,47.0..."
20569,2021S0515019731,Le Sacré-Cur,1,"{""type"":""Point"",""coordinates"":[-69.979863,46.9..."
23733,2021S0515022795,MercierHochelaga-Maisonneuve,1,"{""type"":""Point"",""coordinates"":[-73.5388889,45...."
25319,2021S0515024311,MétabetchouanLac-à-la-Croix,1,"{""type"":""Point"",""coordinates"":[-71.8666667,48...."
29619,2021S0515028429,Port-DanielGascons,1,"{""type"":""Point"",""coordinates"":[-64.9666667,48...."
31289,2021S0515030028,Rivière-des-PrairiesPointe-aux-Trembles,1,"{""type"":""Point"",""coordinates"":[-73.516667,45.65]}"
31432,2021S0515030168,Rock ForestSaint-ÉlieDeauville,1,"{""type"":""Point"",""coordinates"":[-72.0416667,45...."


## Generate GeoJSON file for every dguid
Copy into Cloudflare R2 by running 
```
cd geographies
rclone copy . --transfers 50 --progress cloudflare:/geographies-search
```

In [7]:
if not os.path.exists("geojson"):
    print("Creating DGUID geojson folder")
    os.mkdir("geojson")

for record in geography.to_records():
    dguid = record[1]
    geom = record[-1]
    path = f"geojson/{dguid}.geojson"
    if os.path.exists(path):
        continue
    with open(path, 'w') as geography_fp:
        geography_fp.write(geom)

## Insert data into SQLite database

In [50]:
# Subset of fields to import into SQLite database, add id field as well
geography_subset = geography[['dguid', 'search_name', 'geographic_level']]
geography_subset.insert(0, 'id', geography_subset.index)

cur.executemany("INSERT INTO geographies VALUES(?, ?, ?, ?)", geography_subset.values.tolist())
cur.execute("INSERT INTO geographies_fts(geographies_fts) VALUES ('rebuild')")
con.commit()

### Test out a search query

In [51]:
df = pd.read_sql_query("""
SELECT geographies.dguid, fts.search_name, geographies.geographic_level, rank
FROM geographies_fts AS fts,
     geographies
WHERE fts.search_name MATCH '"Ottawa"*'
AND fts.id = geographies.id
ORDER BY fts.rank, geographies.geographic_level DESC
""", con)
df

,dguid,search_name,geographic_level,rank
0,2021S05003510,Ottawa,10,-9.011603
1,2021A00033506,Ottawa,8,-9.011603
2,2021S05023506008,Ottawa,7,-9.011603
3,2021A00053506008,Ottawa,5,-9.011603
4,2013A000435078,Ottawa--Vanier,4,-9.011603
5,2013A000435075,Ottawa-Centre,4,-9.011603
6,2013A000435079,Ottawa-Ouest--Nepean,4,-9.011603
7,2013A000435077,Ottawa-Sud,4,-9.011603
8,2021S0515026282,Ottawa,1,-9.011603
9,2021S0515026283,Ottawa,1,-9.011603
